### ETL

Realizaremos una limpieza de datos para poder tener un mejor manejo de los datos cuando lleguemos a nuestra visualizacion en PowerBI. Eliminaremos columnas, reemplazaremos valores nulos y vacios con valores estadísticos.

In [4]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.simplefilter("ignore")

In [2]:
# Especifica el nombre del archivo Excel
archivo_excel = '../0-DATA/homicidios.xlsx'

In [3]:
# Especifica los nombres o índices de las hojas que deseas leer
hojas_a_leer = ['HECHOS', 'VICTIMAS']  # Puedes usar los nombres o los índices de las hojas (0-indexed)

In [4]:
# Lee las hojas especificadas en una lista de DataFrames
dataframes = pd.read_excel(archivo_excel, sheet_name=hojas_a_leer)

In [5]:
# Ahora puedes acceder a cada DataFrame por su índice o nombre
dataframe_hoja1 = dataframes['HECHOS']
dataframe_hoja2 = dataframes['VICTIMAS']

In [7]:
dataframe_hoja1.columns

Index(['ID', 'N_VICTIMAS', 'FECHA', 'AAAA', 'MM', 'DD', 'HORA', 'HH',
       'LUGAR_DEL_HECHO', 'TIPO_DE_CALLE', 'Calle', 'Altura', 'Cruce',
       'Dirección Normalizada', 'COMUNA', 'XY (CABA)', 'pos x', 'pos y',
       'PARTICIPANTES', 'VICTIMA', 'ACUSADO'],
      dtype='object')

In [8]:
dataframe_hoja2.columns

Index(['ID_hecho', 'FECHA', 'AAAA', 'MM', 'DD', 'ROL', 'VICTIMA', 'SEXO',
       'EDAD', 'FECHA_FALLECIMIENTO'],
      dtype='object')

Uniendo los dos DF por la columna 'ID' y 'ID_hecho' como podemos visualizar son iguales en cualidad pero no en cantidad, por eso haremos un inner

In [9]:
dataframe_hoja1.ID

0      2016-0001
1      2016-0002
2      2016-0003
3      2016-0004
4      2016-0005
         ...    
691    2021-0093
692    2021-0094
693    2021-0095
694    2021-0096
695    2021-0097
Name: ID, Length: 696, dtype: object

In [10]:
dataframe_hoja2.ID_hecho

0      2016-0001
1      2016-0002
2      2016-0003
3      2016-0004
4      2016-0005
         ...    
712    2021-0092
713    2021-0093
714    2021-0094
715    2021-0095
716    2021-0096
Name: ID_hecho, Length: 717, dtype: object

Unimos con un MERGE

In [11]:
df_merged = pd.merge(dataframe_hoja1, dataframe_hoja2, left_on='ID',right_on='ID_hecho', how='inner')
df_merged

,ID,N_VICTIMAS,FECHA_x,AAAA_x,MM_x,DD_x,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,ID_hecho,FECHA_y,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,...,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,2021-12-13,2021,12,13,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,...,2021-0093,2021-12-13,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
713,2021-0094,1,2021-12-20,2021,12,20,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,...,2021-0094,2021-12-20,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
714,2021-0095,1,2021-12-30,2021,12,30,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,...,2021-0095,2021-12-30,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00
715,2021-0096,1,2021-12-15,2021,12,15,10:30:00,10,AV. EVA PERON 4071,AVENIDA,...,2021-0096,2021-12-15,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20 00:00:00


Analizamos las columnas que tienen los mismo datos

In [12]:
# Función para comparar columnas
def comparar_columnas(df_merged):
    # Obtener el número de filas en el DataFrame
    num_filas = len(df_merged)
    # Iterar sobre las columnas del DataFrame
    for i, col1 in enumerate(df_merged.columns):
        for j, col2 in enumerate(df_merged.columns):
            # No comparar la misma columna
            if i != j:
                # Contar el número de elementos que coinciden en ambas columnas
                coincidencias = (df_merged[col1] == df_merged[col2]).sum()
                # Calcular el porcentaje de coincidencias
                porcentaje_coincidencias = (coincidencias / num_filas) * 100
                # Imprimir resultados
                if porcentaje_coincidencias > 90.0:
                    print(f'Porcentaje de coincidencia entre {col1} y {col2}: {porcentaje_coincidencias:.2f}%')

# Llamar a la función con el DataFrame
comparar_columnas(df_merged)


Porcentaje de coincidencia entre ID y ID_hecho: 100.00%
Porcentaje de coincidencia entre FECHA_x y FECHA_y: 99.72%
Porcentaje de coincidencia entre AAAA_x y AAAA_y: 100.00%
Porcentaje de coincidencia entre MM_x y MM_y: 100.00%
Porcentaje de coincidencia entre DD_x y DD_y: 99.86%
Porcentaje de coincidencia entre VICTIMA_x y VICTIMA_y: 99.44%
Porcentaje de coincidencia entre ID_hecho y ID: 100.00%
Porcentaje de coincidencia entre FECHA_y y FECHA_x: 99.72%
Porcentaje de coincidencia entre AAAA_y y AAAA_x: 100.00%
Porcentaje de coincidencia entre MM_y y MM_x: 100.00%
Porcentaje de coincidencia entre DD_y y DD_x: 99.86%
Porcentaje de coincidencia entre VICTIMA_y y VICTIMA_x: 99.44%


los eliminamos:<br>
* FECHA_x = FECHA_y --> quitaremos los dos ya que en las tres columnas cotiguas tensmos la misma informacion
* AAAA_x = AAAA_y --> quitaremos solo el valor x ya que ambas tienen la misma informacion
* MM_x = MM_y --> quitaremos solo el valor x ya que ambas tienen la misma informacion
* DD_x = DD_y --> quitaremos solo el valor x ya que ambas tienen la misma informacion
* ID = ID_hecho --> quitaremos solo el valor ID_hecho ya que ambas tienen la misma informacion
* VICTIMA_x --> tiene la misma data que victima_y

In [13]:
df_merged = df_merged.drop(columns=["FECHA_x","FECHA_y",'AAAA_x', 'MM_x', 'DD_x', 'ID_hecho',"VICTIMA_x"])
df_merged

,ID,N_VICTIMAS,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,Calle,Altura,Cruce,Dirección Normalizada,...,PARTICIPANTES,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,1,04:00:00,4,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,PIEDRA BUENA AV.,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",...,MOTO-AUTO,AUTO,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,1,01:15:00,1,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,"PAZ, GRAL. AV.",NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",...,AUTO-PASAJEROS,PASAJEROS,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,1,07:00:00,7,AV ENTRE RIOS 2034,AVENIDA,ENTRE RIOS AV.,2034.0,NaN,ENTRE RIOS AV. 2034,...,MOTO-AUTO,AUTO,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,1,00:00:00,0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,LARRAZABAL AV.,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",...,MOTO-SD,SD,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,SD
4,2016-0005,1,05:20:00,5,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,SAN JUAN AV.,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",...,MOTO-PASAJEROS,PASAJEROS,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,17:10:00,17,AV. RIESTRA Y MOM,AVENIDA,RIESTRA AV.,NaN,MOM,RIESTRA AV. y MOM,...,MOTO-AUTO,AUTO,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18,2021-12-18 00:00:00
713,2021-0094,1,01:10:00,1,AU DELLEPIANE Y LACARRA,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",NaN,LACARRA AV.,"DELLEPIANE, LUIS, TTE. GRAL. y LACARRA AV.",...,MOTO-AUTO,AUTO,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43,2021-12-20 00:00:00
714,2021-0095,1,00:43:00,0,AV. GAONA Y TERRADA,AVENIDA,GAONA AV.,NaN,TERRADA,GAONA AV. y TERRADA,...,MOTO-CARGAS,CARGAS,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27,2022-01-02 00:00:00
715,2021-0096,1,10:30:00,10,AV. EVA PERON 4071,AVENIDA,"PERON, EVA AV.",4071.0,NaN,"PERON, EVA AV. 4071",...,AUTO-CARGAS,CARGAS,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60,2021-12-20 00:00:00


#### analizamos columnas parecidas o que no son de relevancia

In [14]:
len(df_merged.Cruce.unique())# eliminamos la columan por que no nos trae una cualidad mas que variables dispersas de donde ocurrio los accidentes

318

In [15]:
len(df_merged["Dirección Normalizada"].unique())#eliminasmo la columna ya que tambien tiene variables dispersas y no nos da un patron

636

In [16]:
len(df_merged["LUGAR_DEL_HECHO"].unique())#eliminasmo la columna ya que tambien tiene variables dispersas y no nos da un patron

683

In [17]:
df_merged["Calle"].value_counts()# como vemos hay un patron repetitivo de accidentes en una calle lo dejamos

Calle
PAZ, GRAL. AV.                                61
RIVADAVIA AV.                                 20
DEL LIBERTADOR AV.                            19
AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI    14
ALBERDI, JUAN BAUTISTA AV.                    13
                                              ..
CERVANTES                                      1
CONGRESO AV.                                   1
ISABEL LA CATOLICA                             1
BILBAO, FRANCISCO AV.                          1
PADRE CARLOS MUJICA                            1
Name: count, Length: 279, dtype: int64

Eliminamos:<br>

* Dirección Normalizada -> eliminasmo la columna ya que tambien tiene variables dispersas y no nos da un patron
* Cruce --> eliminamos la columan por que no nos trae una cualidad mas que variables dispersas de donde ocurrio los accidentes
* LUGAR_DEL_HECHO --> eliminasmo la columna ya que tambien tiene variables dispersas y no nos da un patron
* Altura --> por no tener una cualidad importante y por que el 82% de su data son nulo
* HH = HORA --> no conviene mas tener HH, un numro entero, ya que los minutos no es una metrica muy importante para nuestro analisis
* pos x,pos y = 'XY (CABA)' --> nos conviene tener pos x y pos y ya que esta en formato que lee los programas de visualizacion como PowerBi
* FECHA_FALLECIMIENTO --> tiene una data parecida a la fecha del siniestro, por ende no nos da otra informacion relevante para nuestra presentación
* PARTICIPANTES --> tiene la misma informacion que acusado y victi,a



In [18]:
df_nuevo = df_merged.drop(columns=["Dirección Normalizada","Cruce",'LUGAR_DEL_HECHO','Altura',"HORA",'XY (CABA)',"FECHA_FALLECIMIENTO",'PARTICIPANTES'])
df_nuevo

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD
0,2016-0001,1,4,AVENIDA,PIEDRA BUENA AV.,8,-58.47533969,-34.68757022,AUTO,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,1,1,GRAL PAZ,"PAZ, GRAL. AV.",9,-58.50877521,-34.66977709,PASAJEROS,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,1,7,AVENIDA,ENTRE RIOS AV.,1,-58.39040293,-34.63189362,AUTO,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,1,0,AVENIDA,LARRAZABAL AV.,8,-58.46503904,-34.68092974,SD,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,1,5,AVENIDA,SAN JUAN AV.,1,-58.38718297,-34.62246630,PASAJEROS,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,17,AVENIDA,RIESTRA AV.,7,-58.43353773,-34.64561636,AUTO,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
713,2021-0094,1,1,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",9,-58.46739825,-34.65117757,AUTO,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
714,2021-0095,1,0,AVENIDA,GAONA AV.,11,-58.47293407,-34.61984745,CARGAS,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27
715,2021-0096,1,10,AVENIDA,"PERON, EVA AV.",9,-58.47066794,-34.65021673,CARGAS,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60


Ahora que ya tenemos un dataset mejor estructurado, analizaremos los valores:

    *Faltantes, nulos, vacios, sin datos y tratar de llenarlos lo mejor posible

In [19]:
df_nuevo

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD
0,2016-0001,1,4,AVENIDA,PIEDRA BUENA AV.,8,-58.47533969,-34.68757022,AUTO,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,1,1,GRAL PAZ,"PAZ, GRAL. AV.",9,-58.50877521,-34.66977709,PASAJEROS,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,1,7,AVENIDA,ENTRE RIOS AV.,1,-58.39040293,-34.63189362,AUTO,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,1,0,AVENIDA,LARRAZABAL AV.,8,-58.46503904,-34.68092974,SD,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,1,5,AVENIDA,SAN JUAN AV.,1,-58.38718297,-34.62246630,PASAJEROS,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,17,AVENIDA,RIESTRA AV.,7,-58.43353773,-34.64561636,AUTO,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
713,2021-0094,1,1,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",9,-58.46739825,-34.65117757,AUTO,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
714,2021-0095,1,0,AVENIDA,GAONA AV.,11,-58.47293407,-34.61984745,CARGAS,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27
715,2021-0096,1,10,AVENIDA,"PERON, EVA AV.",9,-58.47066794,-34.65021673,CARGAS,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60


In [20]:
df_nuevo.info()#podemos visualizar que valores nulos no hay solo en la columna 'Calle' pero por DF vemos que existen valores 'SD'

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             717 non-null    object
 1   N_VICTIMAS     717 non-null    int64 
 2   HH             717 non-null    object
 3   TIPO_DE_CALLE  717 non-null    object
 4   Calle          716 non-null    object
 5   COMUNA         717 non-null    int64 
 6   pos x          717 non-null    object
 7   pos y          717 non-null    object
 8   ACUSADO        717 non-null    object
 9   AAAA_y         717 non-null    int64 
 10  MM_y           717 non-null    int64 
 11  DD_y           717 non-null    int64 
 12  ROL            717 non-null    object
 13  VICTIMA_y      717 non-null    object
 14  SEXO           717 non-null    object
 15  EDAD           717 non-null    object
dtypes: int64(5), object(11)
memory usage: 89.8+ KB


In [21]:
for columna in df_nuevo.columns:
    # Contar la cantidad de ocurrencias de "SD" en la columna actual
    cantidad_sd = df_nuevo[columna].value_counts().get('SD', 0)
    print(f"En la columna '{columna}' hay {cantidad_sd} ocurrencias de 'SD'")

En la columna 'ID' hay 0 ocurrencias de 'SD'
En la columna 'N_VICTIMAS' hay 0 ocurrencias de 'SD'
En la columna 'HH' hay 1 ocurrencias de 'SD'
En la columna 'TIPO_DE_CALLE' hay 0 ocurrencias de 'SD'
En la columna 'Calle' hay 0 ocurrencias de 'SD'
En la columna 'COMUNA' hay 0 ocurrencias de 'SD'
En la columna 'pos x' hay 0 ocurrencias de 'SD'
En la columna 'pos y' hay 0 ocurrencias de 'SD'
En la columna 'ACUSADO' hay 23 ocurrencias de 'SD'
En la columna 'AAAA_y' hay 0 ocurrencias de 'SD'
En la columna 'MM_y' hay 0 ocurrencias de 'SD'
En la columna 'DD_y' hay 0 ocurrencias de 'SD'
En la columna 'ROL' hay 11 ocurrencias de 'SD'
En la columna 'VICTIMA_y' hay 9 ocurrencias de 'SD'
En la columna 'SEXO' hay 6 ocurrencias de 'SD'
En la columna 'EDAD' hay 53 ocurrencias de 'SD'


Columna `Calle`:<br>
la columna calle como podemos visualizar en .info() tiene un (1) valor nulo pero 0 valores 'SD'

In [22]:
#vemos donde está la fila con el valor null
df_nuevo[df_nuevo.Calle.isnull()==True]

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD
121,2016-0151,1,20,CALLE,NaN,0,.,.,SD,2016,11,18,PEATON,PEATON,SD,SD


In [23]:
#Llenamos ese dato con el valor de moda, ya que en vez de eliminarlo es mejor tener el dato mas frecuente
valor_mas_comun = df_nuevo['Calle'].mode()[0]
df_nuevo['Calle'] = df_nuevo['Calle'].fillna(valor_mas_comun)
df_nuevo[df_nuevo.ID == '2016-0151']	

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD
121,2016-0151,1,20,CALLE,"PAZ, GRAL. AV.",0,.,.,SD,2016,11,18,PEATON,PEATON,SD,SD


Columna `HH`:<br>
en la columna HH podemos visualizar que tiene un (1) valor 'SD'

In [24]:
#Llenamos ese dato con el calor de moda, ya que en vez de eliminarlo es mejor tener el dato mas frecuente
valor_mas_comun = df_nuevo[df_nuevo['HH'] != 'SD']['HH'].mode()[0]
df_nuevo['HH'] = df_nuevo['HH'].replace('SD', valor_mas_comun)
df_nuevo[df_nuevo.HH == "SD"]

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD


Columna `ACUSADO`:<br>
en la columna ACUSADO podemos visualizar que tiene veintitres (23) valor 'SD'

In [25]:
valor_mas_comun = df_nuevo['ACUSADO'].mode()[0]
df_nuevo['ACUSADO'] = df_nuevo['ACUSADO'].replace('SD', valor_mas_comun)
df_nuevo.ACUSADO.unique()


array(['AUTO', 'PASAJEROS', 'OBJETO FIJO', 'CARGAS', 'MOTO', 'MULTIPLE',
       'OTRO', 'BICICLETA', 'TREN'], dtype=object)

Columna `ROL`:<br>
en la columna ROL podemos visualizar que tiene once (11) valor 'SD'

In [26]:
valor_mas_comun = df_nuevo['ROL'].mode()[0]
df_nuevo['ROL'] = df_nuevo['ROL'].replace('SD', valor_mas_comun)
df_nuevo.ROL.unique()

array(['CONDUCTOR', 'PASAJERO_ACOMPAÑANTE', 'PEATON', 'CICLISTA'],
      dtype=object)

Columna `VICTIMA_y`:<br>
en la columna VICTIMA_y podemos visualizar que tiene nueve (9) valor 'SD'

In [27]:
valor_mas_comun = df_nuevo['VICTIMA_y'].mode()[0]
df_nuevo['VICTIMA_y'] = df_nuevo['VICTIMA_y'].replace('SD', valor_mas_comun)
df_nuevo.VICTIMA_y.unique()

array(['MOTO', 'AUTO', 'PEATON', 'CARGAS', 'BICICLETA', 'PASAJEROS',
       'MOVIL'], dtype=object)

Columna `SEXO`:<br>
en la columna SEXO podemos visualizar que tiene seis (6) valor 'SD'

In [28]:
valor_mas_comun = df_nuevo['SEXO'].mode()[0]
df_nuevo['SEXO'] = df_nuevo['SEXO'].replace('SD', valor_mas_comun)
df_nuevo.SEXO.unique()

array(['MASCULINO', 'FEMENINO'], dtype=object)

Columna `EDAD`:<br>
en la columna EDAD podemos visualizar que tiene cincuenta y tres (53) valor 'SD'

In [29]:
df_nuevo.EDAD.unique()

array([19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20,
       54, 'SD', 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 42, 53, 51, 40,
       87, 76, 75, 35, 80, 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39,
       28, 7, 48, 4, 82, 32, 17, 47, 61, 10, 95, 73, 84, 66, 85, 1, 15,
       13, 77, 44, 31, 62, 74, 71, 11, 86, 69, 72, 12, 63, 92, 68, 91, 64,
       5, 88], dtype=object)

In [30]:
# Convertir los valores numéricos de la columna "EDAD" a tipo int y NaN si no es posible
df_nuevo['EDAD'] = pd.to_numeric(df_nuevo['EDAD'], errors='coerce')

# Calcular el promedio de la columna "EDAD" excluyendo los NaN
promedio_edad = df_nuevo['EDAD'].mean(skipna=True)

# Reemplazar los valores NaN en la columna "EDAD" con el promedio calculado
df_nuevo['EDAD'] = df_nuevo['EDAD'].fillna(round(promedio_edad,0)).astype(int)

df_nuevo.EDAD.unique()

array([19, 70, 30, 18, 29, 22, 16, 59, 65, 34, 41, 50, 38, 21, 52, 36, 20,
       54, 42, 56, 24, 78, 79, 26, 57, 37, 58, 23, 60, 53, 51, 40, 87, 76,
       75, 35, 80, 43, 45, 67, 27, 55, 49, 81, 25, 33, 46, 83, 39, 28,  7,
       48,  4, 82, 32, 17, 47, 61, 10, 95, 73, 84, 66, 85,  1, 15, 13, 77,
       44, 31, 62, 74, 71, 11, 86, 69, 72, 12, 63, 92, 68, 91, 64,  5, 88])

#### Cambiamos los nombre de las columnas

In [31]:
df_nuevo

,ID,N_VICTIMAS,HH,TIPO_DE_CALLE,Calle,COMUNA,pos x,pos y,ACUSADO,AAAA_y,MM_y,DD_y,ROL,VICTIMA_y,SEXO,EDAD
0,2016-0001,1,4,AVENIDA,PIEDRA BUENA AV.,8,-58.47533969,-34.68757022,AUTO,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19
1,2016-0002,1,1,GRAL PAZ,"PAZ, GRAL. AV.",9,-58.50877521,-34.66977709,PASAJEROS,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70
2,2016-0003,1,7,AVENIDA,ENTRE RIOS AV.,1,-58.39040293,-34.63189362,AUTO,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30
3,2016-0004,1,0,AVENIDA,LARRAZABAL AV.,8,-58.46503904,-34.68092974,AUTO,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18
4,2016-0005,1,5,AVENIDA,SAN JUAN AV.,1,-58.38718297,-34.62246630,PASAJEROS,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
712,2021-0093,1,17,AVENIDA,RIESTRA AV.,7,-58.43353773,-34.64561636,AUTO,2021,12,13,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,18
713,2021-0094,1,1,AUTOPISTA,"DELLEPIANE, LUIS, TTE. GRAL.",9,-58.46739825,-34.65117757,AUTO,2021,12,20,PASAJERO_ACOMPAÑANTE,MOTO,FEMENINO,43
714,2021-0095,1,0,AVENIDA,GAONA AV.,11,-58.47293407,-34.61984745,CARGAS,2021,12,30,CONDUCTOR,MOTO,MASCULINO,27
715,2021-0096,1,10,AVENIDA,"PERON, EVA AV.",9,-58.47066794,-34.65021673,CARGAS,2021,12,15,CONDUCTOR,AUTO,MASCULINO,60


In [32]:
df_nuevo.columns

Index(['ID', 'N_VICTIMAS', 'HH', 'TIPO_DE_CALLE', 'Calle', 'COMUNA', 'pos x',
       'pos y', 'ACUSADO', 'AAAA_y', 'MM_y', 'DD_y', 'ROL', 'VICTIMA_y',
       'SEXO', 'EDAD'],
      dtype='object')

In [33]:
nuevos_nombres = {'ID':'Id','N_VICTIMAS':'Victimas','HH':'Hora','TIPO_DE_CALLE':'Lugar','COMUNA':'Comunas','pos x':'Longitud','pos y':'Latitud','ACUSADO':'Acusado','AAAA_y':'Año','MM_y':'Mes','DD_y':'Dia','ROL':'Rol','VICTIMA_y':'Victima','SEXO':'Sexo','EDAD':'Edad'}
df_nuevo = df_nuevo.rename(columns=nuevos_nombres)
df_nuevo.columns

Index(['Id', 'Victimas', 'Hora', 'Lugar', 'Calle', 'Comunas', 'Longitud',
       'Latitud', 'Acusado', 'Año', 'Mes', 'Dia', 'Rol', 'Victima', 'Sexo',
       'Edad'],
      dtype='object')

Guardamos nuestra data limpia

In [34]:
#guardamos en un CSV
df_nuevo.to_csv("../0-DATA/homicidiosETL.csv", index=False, encoding="utf-8") 

In [35]:
#Procedemos de la misma manera guardando en un archivo .parquet
itemsl = pd.read_csv("../0-DATA/homicidiosETL.csv") 
tabla = pa.Table.from_pandas(itemsl) 
pq.write_table(tabla,"../0-DATA/homicidiosETL.parquet") 